In [2]:
import numpy as np
import pandas as pd
import folium
from RMM import *

# 加载待匹配数据
DF = pd.read_csv('test.csv')
GPGGA = DF[DF['data form'].isin(['$GPGGA'])]
RawLat = M2D(np.array(GPGGA['Latitude']))
RawLog = -M2D(np.array(GPGGA['Longitude']))

# 加载路网数据
root = 'train/2021-04-08-US-MTV-1/'
filename = ['GooglePixel4/', 'GooglePixel5/', 'SamsungGalaxyS20Ultra/']
DF = []
for fn in filename:
    path = root + fn + 'ground_truth.csv'
    DF.append(pd.read_csv(path))

Road = pd.concat([DF[0], DF[1], DF[2]]).iloc[:, 2:4]
Road = Road.drop_duplicates().sort_values(by='LatitudeDegrees')
# print(Road )
RoadLat = np.array(Road.iloc[:, 0])
RoadLog = np.array(Road.iloc[:, 1])

out_of_range = []
good = []
to_be_matched = []
lists = []
layer_road = folium.FeatureGroup(name='Road Data')

In [3]:
for i in range(RoadLog.shape[0]):
    folium.Circle(
        radius=5,
        location=(RoadLat[i], RoadLog[i]),
        # 道路数据用蓝色
        color='#0066ff',
        fill=True,
        fill_color='#0066ff'
    ).add_to(layer_road)

In [4]:
layer_unmatched = folium.FeatureGroup(name='Raw Data')
# 对待匹配数据每个点进行查找是否附近在道路附近
for i in range(RawLat.shape[0]):
    for A in range(7, 43, 5):
        flag, index = IsTooLarge(RoadLat, RoadLog, RawLat[i], RawLog[i], A*1e-5)
        if flag and A == 42:
            print('点', i+1, '45米内无道路数据点')
            out_of_range.append(i)
            break
        elif not flag and A <= 7:
            print('点', i+1, '在道路范围7米内')
            good.append(i)
            break
        elif not flag and A > 7:
            print('点', i+1, '距离道路在7米以上')
            to_be_matched.append(i)
            lists.append(index)
            break
        else: continue

# 将各类信息再次显示在地图上
city_map = folium.Map(location=[np.mean(RawLat), np.mean(RawLog)], zoom_start=12)
for i in good:
    folium.Circle(
        radius=5,
        location=(RawLat[i], RawLog[i]),
        # 无需匹配点，使用绿色
        color='#33cc33',
        fill=True,
        fill_color='#33cc33'
    ).add_to(layer_unmatched)

for i in out_of_range:
    folium.Circle(
        radius=5,
        location=(RawLat[i], RawLog[i]),
        # 异常点使用黑色
        color='#000000',
        fill=True,
        fill_color='#000000'
    ).add_to(layer_unmatched)

for i in to_be_matched:
    folium.Circle(
        radius=5,
        location=(RawLat[i], RawLog[i]),
        # 待匹配点使用红色
        color='#ff0000',
        fill=True,
        fill_color='#ff0000'
    ).add_to(layer_unmatched)

点 1 45米内无道路数据点
点 2 45米内无道路数据点
点 3 45米内无道路数据点
点 4 45米内无道路数据点
点 5 45米内无道路数据点
点 6 45米内无道路数据点
点 7 45米内无道路数据点
点 8 45米内无道路数据点
点 9 45米内无道路数据点
点 10 45米内无道路数据点
点 11 45米内无道路数据点
点 12 45米内无道路数据点
点 13 45米内无道路数据点
点 14 45米内无道路数据点
点 15 45米内无道路数据点
点 16 45米内无道路数据点
点 17 45米内无道路数据点
点 18 45米内无道路数据点
点 19 45米内无道路数据点
点 20 45米内无道路数据点
点 21 45米内无道路数据点
点 22 45米内无道路数据点
点 23 45米内无道路数据点
点 24 45米内无道路数据点
点 25 45米内无道路数据点
点 26 45米内无道路数据点
点 27 45米内无道路数据点
点 28 45米内无道路数据点
点 29 45米内无道路数据点
点 30 45米内无道路数据点
点 31 45米内无道路数据点
点 32 45米内无道路数据点
点 33 45米内无道路数据点
点 34 45米内无道路数据点
点 35 45米内无道路数据点
点 36 45米内无道路数据点
点 37 45米内无道路数据点
点 38 45米内无道路数据点
点 39 45米内无道路数据点
点 40 45米内无道路数据点
点 41 45米内无道路数据点
点 42 45米内无道路数据点
点 43 45米内无道路数据点
点 44 45米内无道路数据点
点 45 45米内无道路数据点
点 46 45米内无道路数据点
点 47 45米内无道路数据点
点 48 45米内无道路数据点
点 49 45米内无道路数据点
点 50 45米内无道路数据点
点 51 45米内无道路数据点
点 52 45米内无道路数据点
点 53 45米内无道路数据点
点 54 45米内无道路数据点
点 55 45米内无道路数据点
点 56 45米内无道路数据点
点 57 45米内无道路数据点
点 58 45米内无道路数据点
点 59 45米内无道路数据点
点 60 45米内无道路数据点
点 61 45米内无道路数据点
点 62 45米内无道路数据点
点 63 45米内无道路数据点
点

In [8]:
layer_matched = folium.FeatureGroup(name='Matched Data')
# print(lists)
# 进行匹配
NewLat = []
NewLog = []
print(len(to_be_matched))
for i in range(len(to_be_matched)):
    index = to_be_matched[i]
    road_points = lists[i]
    if len(road_points) > 1:
        try:
            k = (RawLat[index + 1] - RawLat[index - 1]) / (RawLog[index + 1] - RawLog[index - 1])
        except ZeroDivisionError:
            k = (RawLog[index + 1] - RawLog[index - 1]) / (RawLat[index + 1] - RawLat[index - 1])
        j = 0
        while j < len(road_points):
            try:
                k_road = (RoadLat[road_points[j + 1]] - RoadLat[road_points[j]]) / (RoadLog[road_points[j + 1]] - RoadLog[road_points[j]])
            except IndexError:
                if len(road_points) == 1:
                    road_points = []
                    break
                else: break
            except ZeroDivisionError:
                k_road = (RoadLog[road_points[j + 1]] - RoadLog[road_points[j]]) / (RoadLat[road_points[j + 1]] - RoadLat[road_points[j]])

            if abs(k_road - k) <= 0.05:
                j += 1
            else:
                road_points.pop(j)
                # j -= 1

        if len(road_points) != 0:
            NewSinLat = 0
            NewSinLog = 0
            num = 0
            for j in road_points:
                NewSinLat += (1. / dis(RawLog[index], RawLat[index], RoadLog[j], RoadLat[j])) * RoadLat[j]
                NewSinLog += (1. / dis(RawLog[index], RawLat[index], RoadLog[j], RoadLat[j])) * RoadLog[j]
                num += 1. / dis(RawLog[index], RawLat[index], RoadLog[j], RoadLat[j])

            NewSinLat /= num
            NewSinLog /= num
            NewLat.append(NewSinLat)
            NewLog.append(NewSinLog)

    elif len(road_points) == 0:
        pass
    else:
        NewLat.append(RoadLat[road_points[0]])
        NewLog.append(RoadLog[road_points[0]])

print(len(NewLat))

city_map = folium.Map(location=[np.mean(RawLat), np.mean(RawLog)], zoom_start=12)
for i in to_be_matched:
    folium.Circle(
        radius=5,
        location=(RawLat[i], RawLog[i]),
        # 待匹配点使用红色
        color='#ff0000',
        fill=True,
        fill_color='#ff0000'
    ).add_to(layer_matched)
for i in range(len(NewLat)):
    folium.Circle(
        radius=5,
        location=(NewLat[i], NewLog[i]),
        # 待匹配点使用紫红色
        color='#cc33cc',
        fill=True,
        fill_color='#cc33cc'
    ).add_to(layer_matched)

398
398


In [9]:
for i in range(len(to_be_matched)):
    index = to_be_matched[i]
    RawLat[index] = NewLat[i]
    RawLog[index] = NewLog[i]

df = pd.DataFrame({'LatitudeDegree': RawLat, 'LongitudeDegree': RawLog})
df.to_csv('0914.csv')

In [8]:
city_map.add_child(layer_road)
city_map.add_child(layer_unmatched)
city_map.add_child(layer_matched)
folium.LayerControl().add_to(city_map)
city_map.save('test2.html')